___
<img style="float: right; margin: 15px 15px 15px 15px;" src="https://cdn.brandfetch.io/idjblR01Uf/w/400/h/400/theme/dark/icon.jpeg?c=1dxbfHSJFAPEGdCLU4o5B" width="180px" height="180px" />

# <font color= #5aafc1> **PROYECTO FINAL** </font>
#### <font color= #2E9AFE> `Red Aeroportuaria Internacional`</font>
- <Strong> Sofía Maldonado, Viviana Toledo, César Adrián Santos </Strong>
- <Strong> Fecha </Strong>: 17/07/2025

___

In [2]:
# Actualizar 
import importlib
import queries 
importlib.reload(queries)
from queries import DB_Queries

# Establecer conexión
db = DB_Queries('neo5j')

Successfully Connected to Neo4j in neo5j database!
Successfully Connected to GraphDataScience in neo5j database!


# <font color= #5aafc1> **Introducción** </font>

Aquí va la introooo

# <font color= #5aafc1> **Desarrollo** </font>

Añadir atributos de Continente a cada nodo:

In [ ]:
query1 = """MATCH (a:Airport) 
WHERE a.country = 'Burma'
SET a.country = 'Myanmar'
"""

query2 = """MATCH (a:Airport) 
WHERE a.country = 'Congo (Kinshasa)'
SET a.country = 'Democratic Republic of the Congo'
"""

query3 = """MATCH (a:Airport) 
WHERE a.country = 'Congo (Brazzaville)'
SET a.country = 'Republic of the Congo'
"""

query4 = f"""LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/ch0fas/MG_PROYECTO2/main/data/countries.csv' AS row
MATCH (a:Airport)
WHERE a.country = row.Country
SET a.continent = row.Continent
"""

db.custom_query(query1)
db.custom_query(query2)
db.custom_query(query3)
db.custom_query(query4)

Crear subgrafos:

In [5]:
db.create_subgraph('Airport', 'TO', 'NATURAL', 'distance')
db.create_subgraph('Airport', 'TO', 'REVERSE', 'distance')
db.create_subgraph('Airport', 'TO', 'UNDIRECTED', 'distance')

'Subgraph Undirected_Airport already exists. Creation skipped.'

Centrality Algorithms:

In [6]:
db.degrees('Airport')
db.page_rank('Airport')
db.betwenness('Airport')
db.closeness('Airport')

Degree Attribute added Successfully!
In_Degree Attribute added Successfully!
Out_Degree Attribute added Successfully!
Page Rank Attribute added Successfully!
Betweenness Attribute added Successfully!
Closeness Attribute added Successfully!


#### <font color= #5aafc1> **4.2 Exploratory Data Analysis** </font>

Graph Schema:

Node Count:

In [ ]:
# 4.2.2
db.count_nodes('Airport')

,Airports
0,3214


Node Attributes:

In [10]:
db.node_attributes()

,nodeType,nodeLabels,propertyName,propertyTypes,mandatory
0,:`Airport`,[Airport],Id,[Long],True
1,:`Airport`,[Airport],latitude,[Double],True
2,:`Airport`,[Airport],country,[String],True
3,:`Airport`,[Airport],longitude,[Double],True
4,:`Airport`,[Airport],Label,[String],True
5,:`Airport`,[Airport],continent,[String],True


Relationship Count:

In [11]:
db.count_relationships('TO')

,TO_Count
0,36907


Relationship Attributes:

In [12]:
db.rel_attributes()

,relType,propertyName,propertyTypes,mandatory
0,:`TO`,distance,[Double],True


#### <font color= #5aafc1> **4.3** </font>

In [4]:
# Le voy a agregar atributos de latitud y longitud en integer para poder meterlo a Gephi 

query4_3 = f"""
    MATCH (n)
    SET n.latInt = toInteger(n.latitude)
    SET n.longInt = toInteger(n.longitude)
"""

db.custom_query(query4_3)

""


#### <font color= #5aafc1> **4.4** </font>

#### <font color= #5aafc1> **4.5 Elige el nodo de la pregunta 4.4 como origen y uno de los posibles aeropuertos alcanzables de la misma pregunta como destino, muestra la ruta completa del origen al destino en un mapa.** </font>

In [35]:
query4_5 = f""" 
    MATCH(a:Airport {{Label:'Villahermosa'}})-[r1]->(b)-[r2]->(c)-[r3]->(m:Airport {{Label:'Delhi'}})
    WHERE r3.distance IS NOT NULL
    RETURN a.Label,r1.distance AS Dist1,b.Label,r2.distance AS Dist2,c.Label,r3.distance AS Dist3,m.Label
    ORDER BY Dist1+Dist2+Dist3 DESC
    LIMIT 10;
"""

db.custom_query(query4_5)

,a.Label,Dist1,b.Label,Dist2,c.Label,Dist3,m.Label
0,Villahermosa,1358.755582,Houston,10681.305092,Tokyo,5915.459645,Delhi
1,Villahermosa,1358.755582,Houston,11358.558584,Seoul,4657.687448,Delhi
2,Villahermosa,1358.755582,Houston,12947.031367,Doha,2552.069255,Delhi
3,Villahermosa,1358.755582,Houston,11569.256010,Beijing,3813.207294,Delhi
4,Villahermosa,1358.755582,Houston,13139.523542,Dubai,2186.045810,Delhi
5,Villahermosa,678.579085,Mexico City,9209.229377,Paris,6572.730383,Delhi
6,Villahermosa,678.579085,Mexico City,9855.892994,Munich,5906.062233,Delhi
7,Villahermosa,678.579085,Mexico City,9562.676617,Frankfurt,6127.480196,Delhi
8,Villahermosa,678.579085,Mexico City,8913.836258,London,6738.831316,Delhi
9,Villahermosa,678.579085,Mexico City,9216.602463,Amsterdam,6370.585848,Delhi


#### <font color= #5aafc1> **4.6** </font>

#### <font color= #5aafc1> **4.7** </font>

#### <font color= #5aafc1> **4.8. Imagina que vives en el país que elegiste en el punto 4.7 (Colombia) y tienes la posibilidad de viajar a cualquier parte del mundo desde su aeropuerto más importante.** </font>

Primero encontraremos el aeropuerto más importante de Colombia en base a la medida de centralidad _Page Rank_, que evalúa las conexiones que salen del nodo y les asigna una relevancia.

In [14]:
query1 = """MATCH (a:Airport)
WHERE a.country = 'Colombia'
RETURN a.Label AS Airport, a.country AS Country, a.Page_Rank AS Page_Rank
ORDER BY Page_Rank DESC
LIMIT 1
"""

db.custom_query(query1)

,Airport,Country,Page_Rank
0,Bogota,Colombia,9.420864


El aeropuerto más importante de Colombia es el de Bogotá, que se ubica en la capital.

#### <font color= #5aafc1> &nbsp; &nbsp; **4.8.1. ¿Cuántos kilómetros se requieren para visitar las 7 maravillas del mundo?** </font>
#### <font color= #5aafc1> &nbsp; &nbsp; **4.8.2. ¿Cuál sería la mejor ruta para visitarlas todas, en qué orden y por qué?** </font>

Decidimos trabajar con las siguientes 7 maravillas del mundo modernas:

- Giza Pyramids: Cairo
- Great Wall of China: Beijing
- Colosseum: Rome
- Chichen Itzá: Mérida
- Machu Picchu: Cuzco
- Taj Mahal: Lucknow
- Christ the Redeemer: Rio de Janeiro

Para dar respuesta a las preguntas, se analizaron todas las combinaciones posibles desde Bogotá hacia los destinos mencionados. Entonces, a partir de Bogotá, la mejor ruta para recorrer las 7 maravillas del mundo moderno es:

In [70]:
db.seven_wonders_best_route('Airport','Label','Bogota','distance')

,order,distance
0,"(Bogota, Merida, Cuzco, Rio De Janeiro, Rome, Cairo, Lucknow, Beijing)",29588.070071


Esto requiere alrededor de 29,588 km.

#### <font color= #5aafc1> &nbsp; &nbsp; **4.8.3. Aeropuertos visitables desde el Aeropuerto más importante de Colombia en menos de 20,000km** </font>

Crearemos un atributo que nos permita evaluar todas las rutas posibles desde Bogotá:

In [58]:
db.delta_pathing('Airport', 'Label', 'Bogota', 'distance')

Directed Delta Pathing for Bogota already exists. Creation skipped.


,Source,Bridges,Partial_Costs,Total_Cost,Target
0,Bogota,"[Goroka, Port Moresby, Panama City, Nandi, Bogota, Los Angeles]","[0.0, 755.8972047746067, 5606.6853979173375, 14508.500729366064, 17901.63346808019, 18326.70207030612]",18326.702070,Goroka
1,Bogota,"[Madang, Port Moresby, Panama City, Nandi, Bogota, Los Angeles]","[0.0, 755.8972047746067, 5606.6853979173375, 14508.500729366064, 17901.63346808019, 18398.968325213587]",18398.968325,Madang
2,Bogota,"[Mount Hagen, Port Moresby, Panama City, Nandi, Bogota, Los Angeles]","[0.0, 755.8972047746067, 5606.6853979173375, 14508.500729366064, 17901.63346808019, 18417.507717458266]",18417.507717,Mount Hagen


Undirected Delta Pathing for Bogota already exists. Creation skipped.


,Source,Bridges,Partial_Costs,Total_Cost,Target
0,Bogota,"[Goroka, Port Moresby, Panama City, Nandi, Bogota, Los Angeles]","[0.0, 755.8972047746067, 5606.6853979173375, 14508.500729366064, 17901.63346808019, 18326.70207030612]",18326.702070,Goroka
1,Bogota,"[Madang, Port Moresby, Panama City, Nandi, Bogota, Los Angeles]","[0.0, 755.8972047746067, 5606.6853979173375, 14508.500729366064, 17901.63346808019, 18398.968325213587]",18398.968325,Madang
2,Bogota,"[Mount Hagen, Port Moresby, Panama City, Nandi, Bogota, Los Angeles]","[0.0, 755.8972047746067, 5606.6853979173375, 14508.500729366064, 17901.63346808019, 18417.507717458266]",18417.507717,Mount Hagen


Los aeropuertos y rutas visitables que sumen menos de 20,000 km:

In [11]:
query2 = """MATCH (source:Airport) -[r:PATH_Bogota_Directed]-> (target:Airport)
WHERE r.totalCost <= 20_000 
RETURN COUNT(DISTINCT target.Label) AS Distinct_Visitable_Airports
"""
 
db.custom_query(query2)

,Distinct_Visitable_Airports
0,3001


In [55]:
query3 = """MATCH (source:Airport) -[r:PATH_Bogota_Directed]-> (target:Airport)
MATCH (v:Airport)
WHERE r.totalCost <= 20_000 AND id(v) IN r.nodeIds
RETURN source.Label AS Source, target.Label AS Target, COLLECT(v.Label) AS Bridges, 
target.country AS Target_Country, r.totalCost AS Kilometers
LIMIT 20
"""
 
db.custom_query(query3)

,Source,Target,Bridges,Target_Country,Kilometers
0,Bogota,Goroka,"[Goroka, Port Moresby, Panama City, Nandi, Bogota, Los Angeles]",Papua New Guinea,18326.702070
1,Bogota,Madang,"[Madang, Port Moresby, Panama City, Nandi, Bogota, Los Angeles]",Papua New Guinea,18398.968325
2,Bogota,Mount Hagen,"[Mount Hagen, Port Moresby, Panama City, Nandi, Bogota, Los Angeles]",Papua New Guinea,18417.507717
3,Bogota,Nadzab,"[Nadzab, Port Moresby, Panama City, Nandi, Bogota, Los Angeles]",Papua New Guinea,18226.120224
4,Bogota,Port Moresby,"[Port Moresby, Panama City, Nandi, Bogota, Los Angeles]",Papua New Guinea,17901.633468
5,Bogota,Wewak,"[Port Moresby, Wewak, Panama City, Nandi, Bogota, Los Angeles]",Papua New Guinea,18662.934623
6,Bogota,Narssarssuaq,"[Narssarssuaq, Godthaab, Keflavik, Santo Domingo, Bogota, Boston]",Greenland,10016.244656
7,Bogota,Godthaab,"[Godthaab, Keflavik, Santo Domingo, Bogota, Boston]",Greenland,9552.032638
8,Bogota,Sondrestrom,"[Godthaab, Sondrestrom, Keflavik, Santo Domingo, Bogota, Boston]",Greenland,9869.219679
9,Bogota,Thule,"[Godthaab, Thule, Keflavik, Santo Domingo, Bogota, Boston, Ilulissat, Qaanaaq, Upernavik]",Greenland,11300.657141


Algunos de los destinos visitables desde Bogotá a menos de 20,000 km de distancia son Papua New Guinea, Greenland, Iceland y Canada. Las rutas para llegar a cada destino están especificadas en la tabla de arriba, mostrando los países a recorrer.

## <font color= #5aafc1> **Tours** </font>

# Temporal: 7 wonders code

In [ ]:
seven_wonders = ['Cairo', 'Rome', 'Beijing', 'Merida', 'Cuzco', 'Lucknow', 'Rio De Janeiro']
pairs = list(itertools.permutations(seven_wonders,2))

pair_df = pd.DataFrame(columns=['src','dst','middle','distance'])
print(pair_df)

for i in pairs:
    query = f"""
    MATCH (a:Airport {{Label:'{i[0]}'}}), (b:Airport {{Label:'{i[1]}'}})

    call gds.shortestPath.dijkstra.stream('Directed_Airport', {{
        sourceNode: a,
        targetNode: b,
        relationshipWeightProperty: 'distance'
    }})
    YIELD sourceNode, targetNode, totalCost, nodeIds
    RETURN gds.util.asNode(sourceNode).Label AS src, gds.util.asNode(targetNode).Label AS dst, nodeIds AS middle, totalCost AS distance;
    """

    answer = gds.run_cypher(query)
    pair_df = pd.concat([pair_df,answer], sort=False)

pair_df_sorted = pair_df.sort_values(by='distance')
df_final = pair_df_sorted.drop_duplicates(subset=['src','dst'], keep='first')
df_final

In [ ]:
permuts = list(itertools.permutations(seven_wonders))

lookup = {(row.src, row.dst): row.distance for row in df_final.itertuples(index=False)}
print(lookup)

distances = pd.DataFrame(columns=['order','distance'])

for i in permuts:
    pre_distance = 0
    
    for x in range(6):
        par = (i[x], i[x+1])
        dist = lookup.get(par)
        pre_distance += dist

    temp_df = pd.DataFrame({
        "order": [i],
        "distance": pre_distance

    })
    distances = pd.concat([distances,temp_df], sort=False)

distances